# Stage-1 Model Surgery (Adaptive) — Liquid → Hybrid (Net2Net widen + new layers)

This notebook:
- Introspects your student checkpoint to infer d_model, n_layers, and vocab size.
- Widens all tensors that depend on d_model by +10% (rounded to a multiple of 8).
- Appends 2 classic (transformer-style) layers (tiny-out init) and 3 liquid layers (small-scale init).
- Writes out: a new checkpoint, config_stage1.json, freeze_mask.json, and surgery_report.md.

> Fill in the two placeholders for CHECKPOINT_PATH and OUTPUT_DIR below.

In [ ]:
# Cell 1: install (optional if you've already installed requirements.txt)
# This uses the PyTorch cu121 wheel index.
# !pip install -r requirements.txt --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Cell 2: imports, config, and IO paths
import os, re, json, math, time
from dataclasses import dataclass, asdict
from typing import Dict, Any, Tuple, List, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.set_float32_matmul_precision("high")

# ---------------------------
# TODO: set your IO paths
# ---------------------------
CHECKPOINT_PATH = "C:\Users\samsf\Liquid-LLM\Important-Model-Checkpoint\stage0.pt"  # e.g., "/home/jupyter/stage0_best.pt"
OUTPUT_DIR      = "C:\Users\samsf\Liquid-LLM\Important-Model-Checkpoint\stage-1"              # e.g., "/home/jupyter/stage1_surgery_out"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Surgery hyperparameters
WIDTH_PCT        = 1.10  # +10%
ROUND_MULT       = 8     # tensor core multiple
ADD_CLASSIC_LAY  = 2
ADD_LIQUID_LAY   = 3
DEFAULT_MAX_SEQ  = 2048  # safe upper bound; you can change later

In [ ]:
# Cell 3: minimal stub architecture to materialize new param names
# Replace with your package classes later if you want exact runtime parity.
@dataclass
class ModelConfig:
    d_model: int
    n_layers: int
    vocab_size: int
    max_seq_len: int
    layer_types: List[str]  # e.g., ["liquid"]*10 + ["classic"]*2 + ["liquid"]*3

class LiquidBlock(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.ln = nn.LayerNorm(d_model)
        self.fc1 = nn.Linear(d_model, 4*d_model, bias=True)
        self.fc2 = nn.Linear(4*d_model, d_model, bias=True)
        self.scale = nn.Parameter(torch.tensor(1.0))

class ClassicBlock(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.ln = nn.LayerNorm(d_model)
        self.fc1 = nn.Linear(d_model, 4*d_model, bias=True)
        self.fc2 = nn.Linear(4*d_model, d_model, bias=True)
        self.out_scale = nn.Parameter(torch.tensor(1e-3))  # tiny-out

class LiquidLM(nn.Module):
    def __init__(self, cfg: ModelConfig):
        super().__init__()
        self.cfg = cfg
        self.emb = nn.Embedding(cfg.vocab_size, cfg.d_model)
        self.pos_emb = nn.Embedding(cfg.max_seq_len, cfg.d_model)
        layers = []
        for t in cfg.layer_types:
            layers.append(LiquidBlock(cfg.d_model) if t=="liquid" else ClassicBlock(cfg.d_model))
        self.layers = nn.ModuleList(layers)
        self.ln_f = nn.LayerNorm(cfg.d_model)
        self.lm_head = nn.Linear(cfg.d_model, cfg.vocab_size, bias=False)
        # tie weights in this stub
        self.lm_head.weight = self.emb.weight

    def forward(self, *args, **kwargs):
        raise NotImplementedError

In [ ]:
# Cell 4: IO helpers and inference of model stats from state_dict
def load_checkpoint(path: str) -> Dict[str, Any]:
    print(f"Loading checkpoint: {path}")
    sd = torch.load(path, map_location="cpu")
    # accept raw state_dict or {"state_dict": ...}
    if isinstance(sd, dict) and "state_dict" in sd and isinstance(sd["state_dict"], dict):
        return sd["state_dict"]
    if isinstance(sd, dict):
        return sd
    raise ValueError("Unsupported checkpoint format; expected a (dict) state_dict-like payload.")

def infer_layers_from_keys(state_dict: Dict[str, torch.Tensor]) -> int:
    # find all keys like layers.<i>.
    layer_idxs = set()
    import re
    pat = re.compile(r"^layers\.(\d+)\.")
    for k in state_dict.keys():
        m = pat.match(k)
        if m:
            layer_idxs.add(int(m.group(1)))
    return (max(layer_idxs) + 1) if layer_idxs else 0

def infer_d_model(state_dict: Dict[str, torch.Tensor]) -> int:
    # try common 1D params first (LayerNorm weight/bias, etc.)
    candidates = []
    for k, v in state_dict.items():
        if v.dim()==1 and ("ln" in k or "norm" in k or k.endswith("ln_f.weight") or k.endswith("ln_f.bias")):
            candidates.append(v.shape[0])
    if candidates:
        # pick the mode
        from statistics import mode
        try:
            return mode(candidates)
        except:
            return max(set(candidates), key=candidates.count)
    # fallback: use embedding dim if tied
    for k in ["emb.weight", "lm_head.weight"]:
        if k in state_dict and state_dict[k].dim()==2:
            return state_dict[k].shape[1]  # [vocab, d_model]
    # last resort: scan for the most frequent 1D size
    sizes = {}
    for v in state_dict.values():
        if v.dim()==1:
            sizes[v.shape[0]] = sizes.get(v.shape[0], 0) + 1
    if sizes:
        return max(sizes, key=sizes.get)
    raise RuntimeError("Could not infer d_model. Please specify manually.")

def infer_vocab_size(state_dict: Dict[str, torch.Tensor]) -> int:
    # prefer embedding matrix rows
    if "emb.weight" in state_dict and state_dict["emb.weight"].dim()==2:
        return state_dict["emb.weight"].shape[0]
    if "lm_head.weight" in state_dict and state_dict["lm_head.weight"].dim()==2:
        return state_dict["lm_head.weight"].shape[0]
    # fallback heuristic
    return 50257

In [ ]:
# Cell 5: shape tools — figure out which tensor dims depend on d_model and scale them
def round_to_multiple(x: int, mult: int = 8) -> int:
    return int(math.ceil(x / mult) * mult)

def scale_dim_like_dmodel(dim: int, d0: int, d1: int, tol: float = 0.06) -> Optional[int]:
    """
    If 'dim' ~= k*d0 for integer k (k in {1..8}), return k*d1.
    Otherwise None (do not change that dimension).
    tol is relative tolerance.
    """
    if d0 <= 0:
        return None
    ratio = dim / d0
    k = int(round(ratio))
    if k <= 0 or k > 8:
        return None
    if abs(ratio - k) <= tol:
        return k * d1
    return None

def target_shape_from_dmodel(old_shape: Tuple[int, ...], d0: int, d1: int) -> Tuple[int, ...]:
    """
    For 1D or 2D tensors, scale any dimension that looks like a multiple of d_model.
    - For 1D (LayerNorm, biases, vectors): if dim ~= 1*d0, go to d1; if ~= k*d0, go to k*d1.
    - For 2D (Linear weights): scale rows/cols that resemble multiples of d0.
    """
    if len(old_shape)==1:
        dim = old_shape[0]
        new_dim = scale_dim_like_dmodel(dim, d0, d1)
        return (new_dim if new_dim is not None else dim,)
    if len(old_shape)==2:
        o, i = old_shape
        no = scale_dim_like_dmodel(o, d0, d1)
        ni = scale_dim_like_dmodel(i, d0, d1)
        return (no if no is not None else o, ni if ni is not None else i)
    # keep other shapes unchanged
    return old_shape

def widen_tensor(old: torch.Tensor, new_shape: Tuple[int, ...]) -> torch.Tensor:
    """
    Net2Net-style widening by nearest-neighbor sampling + small jitter.
    Supports 1D and 2D tensors.
    """
    if tuple(old.shape) == tuple(new_shape):
        return old.clone()
    g = torch.Generator().manual_seed(17)
    if old.dim()==1:
        old_n = old.shape[0]; new_n = new_shape[0]
        idx = torch.linspace(0, old_n-1, new_n).round().long()
        out = old[idx]
        std = out.std()
        jitter = (std if std > 0 else 1e-3) * 0.01
        out = out + torch.randn_like(out, generator=g) * jitter
        return out
    if old.dim()==2:
        o, i = old.shape
        no, ni = new_shape
        ridx = torch.linspace(0, o-1, no).round().long()
        cidx = torch.linspace(0, i-1, ni).round().long()
        out = old[ridx][:, cidx]
        std = out.std()
        jitter = (std if std > 0 else 1e-3) * 0.01
        out = out + torch.randn_like(out, generator=g) * jitter
        return out
    # other ranks: return copy unmodified
    return old.clone()

def tiny_out_init_like(t: torch.Tensor, scale: float=1e-3) -> torch.Tensor:
    out = torch.empty_like(t)
    if t.dim()==2:
        nn.init.kaiming_uniform_(out, a=math.sqrt(5))
        out.mul_(scale)
    elif t.dim()==1:
        nn.init.zeros_(out)
    else:
        out.copy_(t)
    return out

In [ ]:
# Cell 6: perform surgery (adaptive to unknown layer details)
with torch.no_grad():
    sd0 = load_checkpoint(CHECKPOINT_PATH)

    n0 = infer_layers_from_keys(sd0)
    d0 = infer_d_model(sd0)
    vsz = infer_vocab_size(sd0)

    print(f"Inferred Stage-0: n_layers={n0}, d_model={d0}, vocab_size={vsz}")

    # compute widened d_model
    d1_raw = int(math.ceil(WIDTH_PCT * d0))
    d1 = round_to_multiple(d1_raw, ROUND_MULT)
    print(f"Target d_model: {d0} -> {d1} (requested +10% -> rounded to /{ROUND_MULT})")

    layer_types = ["liquid"]*n0 + ["classic"]*ADD_CLASSIC_LAY + ["liquid"]*ADD_LIQUID_LAY
    n1 = len(layer_types)

    # build a destination model (stub) to get target keyset and shapes
    cfg1 = ModelConfig(
        d_model=d1,
        n_layers=n1,
        vocab_size=vsz,
        max_seq_len=DEFAULT_MAX_SEQ,
        layer_types=layer_types
    )
    model1 = LiquidLM(cfg1).to("cpu")
    sd1 = model1.state_dict()

    # Pass 1: copy/widen all tensors that exist in both models by shape logic
    copied, widened, skipped = [], [], []

    for k, v_new in sd1.items():
        if k in sd0:
            v_old = sd0[k]
            if v_old.shape == v_new.shape:
                sd1[k] = v_old.clone()
                copied.append(k)
            else:
                # compute target by scaling dims that depend on d_model
                tgt_shape = target_shape_from_dmodel(v_old.shape, d0, d1)
                if tuple(tgt_shape) == tuple(v_new.shape):
                    sd1[k] = widen_tensor(v_old, v_new.shape)
                    widened.append((k, v_old.shape, v_new.shape))
                else:
                    # leave new init for newly added/changed layers
                    skipped.append((k, v_old.shape, v_new.shape))
        else:
            # brand new param (e.g., added layers) — keep model1 init
            skipped.append((k, None, v_new.shape))

    # Pass 2: specialize inits for the appended layers
    # classic layers: tiny-out init on fc weights; LN defaults; out_scale=1e-3
    for j in range(n0, n0+ADD_CLASSIC_LAY):
        prefix = f"layers.{j}."
        for sub in ["ln.weight","ln.bias","fc1.weight","fc1.bias","fc2.weight","fc2.bias","out_scale"]:
            k = prefix+sub
            if k in sd1:
                if sub in ("fc1.weight","fc2.weight"):
                    sd1[k] = tiny_out_init_like(sd1[k], scale=1e-3)
                elif sub in ("fc1.bias","fc2.bias"):
                    sd1[k] = torch.zeros_like(sd1[k])
                elif sub=="ln.weight":
                    sd1[k] = torch.ones_like(sd1[k])
                elif sub=="ln.bias":
                    sd1[k] = torch.zeros_like(sd1[k])
                elif sub=="out_scale":
                    sd1[k] = torch.full_like(sd1[k], 1e-3)

    # liquid layers (new): small-scale mlp weights; LN defaults; scale ~1e-3
    for j in range(n0+ADD_CLASSIC_LAY, n1):
        prefix = f"layers.{j}."
        for sub in ["ln.weight","ln.bias","fc1.weight","fc1.bias","fc2.weight","fc2.bias","scale"]:
            k = prefix+sub
            if k in sd1:
                if sub in ("fc1.weight","fc2.weight"):
                    nn.init.kaiming_uniform_(sd1[k], a=math.sqrt(5))
                    sd1[k].mul_(1e-3)
                elif sub in ("fc1.bias","fc2.bias"):
                    sd1[k] = torch.zeros_like(sd1[k])
                elif sub=="ln.weight":
                    sd1[k] = torch.ones_like(sd1[k])
                elif sub=="ln.bias":
                    sd1[k] = torch.zeros_like(sd1[k])
                elif sub=="scale":
                    sd1[k] = torch.tensor(1e-3)

    # Sanity: load with strict=False to allow benign key mismatches
    missing, unexpected = model1.load_state_dict(sd1, strict=False)
    print("Loaded into destination model with strict=False.")
    print("Missing keys:", len(missing))
    print("Unexpected keys:", len(unexpected))

    # Outputs
    ts       = int(time.time())
    out_ckpt = os.path.join(OUTPUT_DIR, f"stage1_surgery_{ts}.pt")
    out_cfg  = os.path.join(OUTPUT_DIR, "config_stage1.json")
    out_mask = os.path.join(OUTPUT_DIR, "freeze_mask.json")
    out_rep  = os.path.join(OUTPUT_DIR, "surgery_report.md")

    # Freeze mask: mark classic layers for “freeze after Stage-1”
    freeze_mask = {f"layers.{i}": (layer_types[i] == "classic") for i in range(n1)}

    torch.save({"state_dict": sd1, "note": "Stage-1 surgery checkpoint"}, out_ckpt)
    with open(out_cfg, "w") as f: json.dump(asdict(cfg1), f, indent=2)
    with open(out_mask, "w") as f: json.dump(freeze_mask, f, indent=2)

    with open(out_rep, "w") as f:
        f.write(
f"""# Stage-1 Surgery Report

Source checkpoint: `{CHECKPOINT_PATH}`
Output checkpoint: `{out_ckpt}`

Inferred (Stage-0):
- n_layers: {n0}
- d_model:  {d0}
- vocab:    {vsz}

Stage-1 target:
- d_model:  {d0} -> {d1} (WIDTH_PCT={WIDTH_PCT}, rounded /{ROUND_MULT})
- layers:   {n0} -> {n1}
- layout:   {layer_types}

Copy stats:
- copied (same-shape): {len(copied)}
- widened (Net2Net):   {len(widened)}
- kept/init new:       {len(skipped)}

Missing keys on load:   {len(missing)}
Unexpected keys on load:{len(unexpected)}

Notes:
- Classic layers are marked True in freeze_mask.json for freezing AFTER Stage-1.
- This notebook adapts tensor shapes by detecting dims that are ~= k*d_model (k in 1..8).
"""
        )
    print("\nWROTE:")
    print("-", out_ckpt)
    print("-", out_cfg)
    print("-", out_mask)
    print("-", out_rep)